In [1]:
import yfinance as yf
import pandas as pd
from ipywidgets import Output
from sklearn.metrics import precision_score, accuracy_score
from sklearn.metrics import precision_score, accuracy_score
predictors = []

In [2]:
stocks = ["^gspc"]
data =pd.DataFrame([])
pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
first = True
               
for stock in stocks:
    stk = yf.Ticker(stock)  
    stock_data = stk.history(period='max') 
    stock_data=stock_data.loc["1990-1-1":].copy()
    del stock_data["Dividends"]
    del stock_data["Stock Splits"] 
    del stock_data["Open"]
    del stock_data["High"]
    del stock_data["Low"]
    stock_data["Tomarrow"] = stock_data["Close"].shift(-1)
    stock_data["Target"] = (stock_data["Close"]<stock_data["Tomarrow"]).astype(int)
    
    horizons = [2,5,60,250,1000]
    # predictors = []

    for horizon in horizons:
        rolling_averages = stock_data.rolling(horizon).mean()

        ratio_column = f"{stock}_Close_Avg_{horizon}"
        stock_data[ratio_column] = stock_data["Close"] / rolling_averages["Close"]

        trend_column = f"{stock}_Trend_{horizon}"
        stock_data[trend_column] = stock_data.shift(1).rolling(horizon).sum()["Target"]
        
        v_ratio_col =f"{stock}_Vol_Avg_{horizon}"
        stock_data[v_ratio_col] = stock_data["Volume"]/rolling_averages["Volume"]
        
        predictors+= [ratio_column, trend_column]
    
    if  first:
        first = False
    else:
        print("delete pred name")
        del stock_data['Close']
        del stock_data['Volume']
        del stock_data['Target']
    data = pd.concat([data,stock_data],ignore_index = False,axis=1)
    

In [3]:
data=data.dropna()

In [4]:
import sklearn
from sklearn.ensemble import RandomForestClassifier

train = data.iloc[:-100]
test = data.iloc[-100:]

In [5]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [6]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [7]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [8]:
predictions = backtest(data, model, predictors)

In [9]:
predictions["Predictions"].value_counts()

Predictions
0.00    4280
1.00     830
Name: count, dtype: int64

In [10]:
predictions["Target"].value_counts() / predictions.shape[0]

Target
1   0.54
0   0.46
Name: count, dtype: float64

In [11]:
prec = precision_score(predictions["Target"], predictions["Predictions"])
acc = accuracy_score(predictions["Target"], predictions["Predictions"])

In [12]:
print('precision: ',prec)
print('accuracy: ',acc)

precision:  0.5734939759036145
accuracy:  0.47906066536203523


In [13]:
predictions

,Target,Predictions
Date,,
2003-11-14 00:00:00-05:00,0,0.00
2003-11-17 00:00:00-05:00,0,1.00
2003-11-18 00:00:00-05:00,1,1.00
2003-11-19 00:00:00-05:00,0,0.00
2003-11-20 00:00:00-05:00,1,1.00
...,...,...
2024-02-29 00:00:00-05:00,1,0.00
2024-03-01 00:00:00-05:00,0,0.00
2024-03-04 00:00:00-05:00,0,0.00


In [14]:
feature_imp=pd.Series( model.feature_importances_).sort_index(ascending=True)

In [15]:
pred=pd.Series(predictors)

In [16]:
print(pd.concat([pred,feature_imp],axis=1))

                      0    1
0     ^gspc_Close_Avg_2 0.16
1         ^gspc_Trend_2 0.01
2     ^gspc_Close_Avg_5 0.15
3         ^gspc_Trend_5 0.03
4    ^gspc_Close_Avg_60 0.15
5        ^gspc_Trend_60 0.06
6   ^gspc_Close_Avg_250 0.14
7       ^gspc_Trend_250 0.08
8  ^gspc_Close_Avg_1000 0.13
9      ^gspc_Trend_1000 0.08
